In [1]:
import boto3
import pandas as pd
import psycopg2 #for redshift port
import json

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('Desktop\semester 4\capstoneproject\cluster.config'))

In [3]:
KEY=              config.get("AWS","KEY")    
SECRET=           config.get("AWS","SECRET")

DWH_CLUSTER_TYPE= config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES=    config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE=    config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_INDENTIFIER= config.get("DWH","DWH_CLUSTER_INDENTIFIER")
DWH_DB=         config.get("DWH","DWH_DB")
DWH_DB_USER=    config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD=config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT=       config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME= config.get("DWH","DWH_IAM_ROLE_NAME")

(DWH_DB_USER,DWH_DB_PASSWORD,DWH_DB)

('awsuser', 'Password123', 'mydb')

In [4]:
ec2=boto3.resource('ec2',
                   region_name="ca-central-1",
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                  )

s3=boto3.resource('s3',
                   region_name="ca-central-1",
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                  )

iam=boto3.client('iam',
                   region_name="ca-central-1",
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                  )

redshift=boto3.client('redshift',
                   region_name="ca-central-1",
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                  )

In [5]:
bucket=s3.Bucket("pypipe")
log_files=[filename.key for filename in bucket.objects.filter(Prefix='')]
log_files

['allevents_pipe.txt',
 'allusers_pipe.txt',
 'category_pipe.txt',
 'date2008_pipe.txt',
 'listings_pipe.txt',
 'sales_tab.txt',
 'venue_pipe.txt']

In [6]:
roleArn= iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
roleArn

'arn:aws:iam::008925947455:role/redhift_s3'

In [7]:
#creating redshift cluster
try:
    response = redshift.create_cluster(
    ClusterType=DWH_CLUSTER_TYPE,
    NodeType=DWH_NODE_TYPE,
        
    DBName=DWH_DB,
    ClusterIdentifier=DWH_CLUSTER_INDENTIFIER,
    MasterUsername=DWH_DB_USER,
    MasterUserPassword=DWH_DB_PASSWORD,
        
    IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

In [12]:
redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_INDENTIFIER)

{'Clusters': [{'ClusterIdentifier': 'my-first-redshift',
   'NodeType': 'dc2.large',
   'ClusterStatus': 'available',
   'ClusterAvailabilityStatus': 'Available',
   'MasterUsername': 'awsuser',
   'DBName': 'mydb',
   'Endpoint': {'Address': 'my-first-redshift.cmznl1jdfzkp.ca-central-1.redshift.amazonaws.com',
    'Port': 5439},
   'ClusterCreateTime': datetime.datetime(2023, 2, 21, 23, 2, 9, 42000, tzinfo=tzutc()),
   'AutomatedSnapshotRetentionPeriod': 1,
   'ManualSnapshotRetentionPeriod': -1,
   'ClusterSecurityGroups': [],
   'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0333cc4e835a66dab',
     'Status': 'active'}],
   'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
     'ParameterApplyStatus': 'in-sync'}],
   'ClusterSubnetGroupName': 'default',
   'VpcId': 'vpc-054afb094c646333c',
   'AvailabilityZone': 'ca-central-1b',
   'PreferredMaintenanceWindow': 'sun:16:30-sun:17:00',
   'PendingModifiedValues': {},
   'ClusterVersion': '1.0',
   'AllowVersi

In [13]:
def prettyRedShiftProps(props):
    pd.set_option('display.max_colwidth',-1)
    keysToShow = ["ClusterIdentifier","NodeType","ClusterStatus","MasterUsername","DBName","Endpoint",'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x,columns=["key","Value"])
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_INDENTIFIER)['Clusters'][0]
prettyRedShiftProps(myClusterProps)

C:\Users\Mirul\AppData\Local\Temp\ipykernel_18092\415864532.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth',-1)


,key,Value
0,ClusterIdentifier,my-first-redshift
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,awsuser
4,DBName,mydb
5,Endpoint,"{'Address': 'my-first-redshift.cmznl1jdfzkp.ca-central-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-054afb094c646333c


In [14]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
DB_NAME = myClusterProps['DBName']
DB_USER = myClusterProps['MasterUsername']

In [15]:
try:
    vpc=ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    #authorizing all incoming traffic
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-01075f86b4ee96aaf')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


In [16]:
try:
    con= psycopg2.connect(host=DWH_ENDPOINT,dbname=DB_NAME,user=DB_USER,password="Password123",port=5439)
except psycopg2.Error as e:
    print("Error: Could not make connection to the database")
    print(e)
        
con.set_session(autocommit=True)

In [17]:
try:
    cur=con.cursor()
except psycopg2.Error as e:
    print("Error: Coul not get cursor to the database")
    print(e)

In [18]:
#sortkey divides the data by some range for query optimization
#dist key has three types dist key for large fact tables
#Distribution all(distribute whole table  in all nodes) for medium dimensions
#Distribution even (distribute evenly)
try:
    cur.execute("""create table users(
    userid integer not null distkey sortkey,
    username char(8),
    firstname varchar(30),
    lastname varchar(30),
    city varchar(30),
    state char(2),
    email varchar(100),
    phone char(14),
    likesports boolean,
    liketheatre boolean,
    likeconcerts boolean,
    likejazz boolean,
    likeclassical boolean,
    likeopera boolean,
    likerock boolean,
    likevegas boolean,
    likebroadway boolean,
    likemusicals boolean
    );""")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [20]:
try:
    cur.execute("""create table venue(
    venueid smallint not null distkey sortkey,
    venuename varchar(100),
    venuecity varchar(30),
    venuestate char(2),
    venueseats integer
    );""")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [22]:
try:
    cur.execute("""create table category(
    catid smallint not null distkey sortkey,
    catgroup varchar(10),
    catrname varchar(10),
    catdesc varchar(50)
    );
    
    create table date(
    dateid smallint not null distkey sortkey,
    caldate date not null,
    day character(3) not null,
    week smallint not null,
    month character(5) not null,
    qtr character(5) not null,
    year smallint not null,
    holiday boolean default('N')
    );
    
    create table event(
    eventid integer not null distkey,
    venueid smallint not null,
    catid smallint not null,
    dateid smallint not null sortkey,
    eventname varchar(200),
    starttime timestamp
    );
    
    create table listing(
    listid integer not null distkey,
    sellerid integer not null,
    eventid integer not null,
    dateid smallint not null sortkey,
    numtickets smallint not null,
    priceperticket decimal(8,2),
    totalprice decimal(8,2),
    listtime timestamp
    );
    
    """)
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [24]:
#copy command to copy data from s3 to redshift table
try:
    cur.execute("""
    copy users from 's3://pypipe/allusers_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::008925947455:role/redhift_s3'
    delimiter '|'
    region 'ca-central-1'
    """)
except psycopg2.Error as e:
    print("Error: Issue copying table")
    print(e)

In [27]:
#copy command to copy data from s3 to redshift table
try:
    cur.execute("""
    copy venue from 's3://pypipe/venue_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::008925947455:role/redhift_s3'
    delimiter '|'
    region 'ca-central-1'    
    """)
    
except psycopg2.Error as e:
    print("Error: Issue copying table")
    print(e)

In [28]:
#copy command to copy data from s3 to redshift table
try:
    cur.execute(""" 
    copy category from 's3://pypipe/category_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::008925947455:role/redhift_s3'
    delimiter '|'
    region 'ca-central-1'
    
    """)
    
except psycopg2.Error as e:
    print("Error: Issue copying table")
    print(e)


In [29]:
#copy command to copy data from s3 to redshift table
try:
    cur.execute(""" 
    
    copy date from 's3://pypipe/date2008_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::008925947455:role/redhift_s3'
    delimiter '|'
    region 'ca-central-1'
    
    """)
    
except psycopg2.Error as e:
    print("Error: Issue copying table")
    print(e)

In [30]:
#copy command to copy data from s3 to redshift table
try:
    cur.execute(""" 
    
    copy event from 's3://pypipe/allevents_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::008925947455:role/redhift_s3'
    delimiter '|'
    region 'ca-central-1'
    """)
    
except psycopg2.Error as e:
    print("Error: Issue copying table")
    print(e)

In [32]:
#copy command to copy data from s3 to redshift table
try:
    cur.execute(""" 
    copy listing from 's3://pypipe/listings_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::008925947455:role/redhift_s3'
    delimiter '|'
    region 'ca-central-1'
    """)
    
except psycopg2.Error as e:
    print("Error: Issue copying table")
    print(e)

In [36]:
#copy command to copy data from s3 to redshift table
try:
    cur.execute(""" 
    select * from users;
    """)
    row=cur.fetchone()
    while row:
        print(row)
        row=cur.fetchone()
        break
except psycopg2.Error as e:
    print("Error: Issue copying table")
    print(e)

(2, 'PGL08LJI', 'Vladimir', 'Humphrey', 'Murfreesboro', 'SK', 'Suspendisse.tristique@nonnisiAenean.edu', '(783) 492-1886', None, None, None, True, True, None, None, True, False, True)


In [37]:
try:
    con.close()
except psycopg2.Error as e:
    print("Error: Could not make connection to the database")
    print(e)

In [39]:
redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_INDENTIFIER, SkipFinalClusterSnapshot=True)

{'Cluster': {'ClusterIdentifier': 'my-first-redshift',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'awsuser',
  'DBName': 'mydb',
  'Endpoint': {'Address': 'my-first-redshift.cmznl1jdfzkp.ca-central-1.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2023, 2, 21, 23, 2, 9, 42000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0333cc4e835a66dab',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-054afb094c646333c',
  'AvailabilityZone': 'ca-central-1b',
  'PreferredMaintenanceWindow': 'sun:16:30-sun:17:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'Numb

In [ ]:
#connected redshift cluster to Amazon Quicksight and performed visualizations 